In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.transforms as transforms
# Varsayalım ki 'datasets' ve 'dataloaders' zaten tanımlı
# from your_data_module import train_dataloader, num_classes

# Yeni görevimiz için sınıf sayısı (Örn: 5 farklı çiçek türü)
num_classes = 5

In [2]:
## Modeli Yükleme
# Önceden eğitilmiş ResNet-18 modelini yüklüyoruz
model_ft = models.resnet18(weights='IMAGENET1K_V1')

print("--- Katman Dondurma Başlatılıyor ---")
# 🧱 A. Katmanları Dondurma (Erken Katmanlar)
# Tüm parametrelerin 'requires_grad' özelliğini False yaparak donduruyoruz.
# Bu, PyTorch'a bu ağırlıkların geri yayılım (backpropagation) sırasında
# güncellenmemesi gerektiğini söyler.

for param in model_ft.parameters():
    param.requires_grad = False

print("Tüm orijinal katmanlar donduruldu (weights sabit).")

## ⚙️ B. Çıkış Katmanını Değiştirme ve Çözme (Unfreezing)

# ResNet'in son tam bağlantılı (Fully Connected / FC) katmanının adı 'fc'dir.
# Bu katman, orijinal görevdeki sınıf sayısına göre ayarlanmıştır.
# Yeni görevimiz için bu katmanı sıfırdan oluşturuyoruz.
num_ftrs = model_ft.fc.in_features # Giriş özellik sayısını al

# Yeni, rastgele başlatılmış bir tam bağlantılı katman tanımlıyoruz.
# Bu yeni katmanın ağırlıkları RASTGELE başlar ve default olarak 'requires_grad=True' olur,
# yani eğitim sırasında güncellenecektir (Çözüldü/Unfrozen).
model_ft.fc = nn.Linear(num_ftrs, num_classes)

print(f"Çıkış katmanı {num_ftrs} -> {num_classes} olarak güncellendi.")
print("--- Katman Dondurma Tamamlandı ---")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]


--- Katman Dondurma Başlatılıyor ---
Tüm orijinal katmanlar donduruldu (weights sabit).
Çıkış katmanı 512 -> 5 olarak güncellendi.
--- Katman Dondurma Tamamlandı ---


In [4]:
# 🔍 Yalnızca 'requires_grad=True' olan (yani çözülmüş) parametreleri alıyoruz.
# Bu durumda bu, sadece yeni 'model_ft.fc' katmanının parametreleridir.
params_to_update = []
for name, param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        # print(f"Güncellenecek Parametre: {name}")

# 📉 Hiperparametre: Düşük Öğrenme Oranı (Genellikle 1e-4 veya 1e-3 kullanılır)
# Modelin mevcut ağırlıklarını bozmamak için sıfırdan eğitime göre çok daha düşük bir değer seçiyoruz.
fine_tune_learning_rate = 0.001

# Optimizer Tanımlama (Adam yaygın olarak kullanılır)
optimizer_ft = optim.Adam(params_to_update, lr=fine_tune_learning_rate)

print(f"Hiperparametre (Öğrenme Oranı) ayarlandı: {fine_tune_learning_rate}")

Hiperparametre (Öğrenme Oranı) ayarlandı: 0.001


In [5]:
# 🔍 Yalnızca 'requires_grad=True' olan (yani çözülmüş) parametreleri alıyoruz.
# Bu durumda bu, sadece yeni 'model_ft.fc' katmanının parametreleridir.
params_to_update = []
for name, param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        # print(f"Güncellenecek Parametre: {name}")

# 📉 Hiperparametre: Düşük Öğrenme Oranı (Genellikle 1e-4 veya 1e-3 kullanılır)
# Modelin mevcut ağırlıklarını bozmamak için sıfırdan eğitime göre çok daha düşük bir değer seçiyoruz.
fine_tune_learning_rate = 0.001

# Optimizer Tanımlama (Adam yaygın olarak kullanılır)
optimizer_ft = optim.Adam(params_to_update, lr=fine_tune_learning_rate)

print(f"Hiperparametre (Öğrenme Oranı) ayarlandı: {fine_tune_learning_rate}")

Hiperparametre (Öğrenme Oranı) ayarlandı: 0.001


In [11]:
# 🎯 Hiperparametre: Düşük Epoch Sayısı (Genellikle 5'ten az)
num_epochs = 3
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

print(f"Eğitim cihazı: {device}")
print(f"Hiperparametre (Epoch Sayısı) ayarlandı: {num_epochs}")

# --- Basitleştirilmiş Eğitim Adımları ---
# (Burada veri yükleyicilerinin (dataloaders) ve döngünün tanımlandığını varsayalım)

# for epoch in range(num_epochs):
#     model_ft.train() # Modeli eğitim moduna al
#     running_loss = 0.0
#     for inputs, labels in train_dataloader: # Varsayılan veri yükleyici
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optimizer_ft.zero_grad() # Gradyanları sıfırla

#         # İleri yayılım (Forward Pass)
#         outputs = model_ft(inputs)
#         loss = criterion(outputs, labels)

#         # Geri yayılım (Backpropagation) ve Ağırlık Güncelleme (Sadece çözülmüş katmanlar güncellenir!)
#         loss.backward()
#         optimizer_ft.step()

#         running_loss += loss.item() * inputs.size(0)
#
#     epoch_loss = running_loss / len(train_dataset)
#     print(f"Epoch {epoch+1}/{num_epochs}, Kayıp (Loss): {epoch_loss:.4f}")

# print("Fine-Tuning Eğitimi Tamamlandı!")

Eğitim cihazı: cpu
Hiperparametre (Epoch Sayısı) ayarlandı: 3
